## create the model

In [1]:
import torch
from narnian.model import Model
from narnian.attributes import Attributes
# from modules.networks import PredRNN
from modules.networks import GenCTBEInitStateBZeroInput, GenRNN

In [6]:
class DemoModel(Model):

    def __init__(self, attributes: list[Attributes], lr: float = 0.0001, device: torch.device = torch.device("cpu")):
        """Creates a model composed of a generator and a predictor."""

        # getting shape info from attributes (it is needed to build the generator/predictor)
        assert len(attributes) == 2, "Only two attributes are supported/expected (about y and d)"
        u_shape = attributes[0].shape
        d_dim = attributes[1].shape.numel()
        y_dim = attributes[0].shape.numel()
        
        generator = GenRNN(u_shape=u_shape, d_dim=d_dim, y_dim=y_dim, h_dim=150)
        # generator = GenCTBEInitStateBZeroInput(u_shape=u_shape, d_dim=d_dim, y_dim=y_dim, h_dim=150, delta=0.1,
        #                                        local=True, cnu_memories=0)
        predictor = None

        # creating the model (superclass)
        super(DemoModel, self).__init__(generator, predictor, attributes, device=device)

        # extra stuff
        self.optim = torch.optim.SGD(list(self.generator.parameters()), lr=lr)
        self.loss_gen = torch.nn.functional.mse_loss
        self.loss_pred = torch.nn.functional.mse_loss

    def learn(self,
              y: torch.Tensor | None, yhat: torch.Tensor | None,
              d: torch.Tensor | None, dhat: torch.Tensor | None) \
            -> tuple[float, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None]:
        """Learn from different types of data (some of them could be None)."""

        # clean arguments, ensuring that what should be forced to None is actually forced to None
        _, y, yhat, d, dhat = super().learn(y, yhat, d, dhat)  # it seems unuseful, but IT MUST be called!

        # evaluating loss function
        loss = ((self.loss_gen(y, yhat) if yhat is not None else 0.) +
                (self.loss_pred(d, dhat) if dhat is not None else 0.))

        # learning
        self.optim.zero_grad()
        loss_as_float = loss.item()
        loss.backward()
        self.optim.step()

        return loss_as_float, y, yhat, d, dhat

## learn and generate signal
In this Sandbox, the Teacher records a 2000 samples 

In [3]:
import torch
from narnian.server import Server
from narnian.streams import Stream
from narnian.model import EmptyModel
from basic.basic_agent import BasicAgent
from basic.basic_streams import Sin, Square
from basic.basic_environment import BasicEnvironment

In [7]:
# creating environment
env = BasicEnvironment("env", title="Demo Sandbox Signal")
device = torch.device("cpu")

# adding streams to the environment
env.add_stream(Stream.create(name="sin", creator=env.name, stream=Sin(freq=0.06, phase=0.5, delta=0.1)))

# modeling behaviour of the environment
env.add_transit("init", "streams_enabled", action="enable_all_streams")
env.add_transit("streams_enabled", "streams_sent", action="send_streams_to_all")
env.add_transit("streams_sent", "ready", action="send_agents_to_all")

# creating teacher agent
ag = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "recording1", action="record", args={"stream_hash": env.name + ":sin", "steps": 2000})
ag.add_transit("recording1", "playlist_ready", action="set_pref_streams",
               args={"stream_hashes": [ag.name + ":recorded1"], "repeat": 1})
ag.add_state_action("playlist_ready", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})
ag.add_transit("playlist_ready", "student_found", action="send_engagement")
ag.add_transit("student_found", "playlist_ready", action="nop")
ag.add_transit("student_found", "student_engaged", action="got_engagement")
ag.add_transit("student_engaged", "stream_shared", action="share_streams")
ag.add_transit("stream_shared", "asked_learn", action="ask_learn_gen",
               args={"du_hash": "<playlist>", "yhat_hash": "<playlist>", "dhat_hash": "<playlist>", "ask_steps": 2000})
ag.add_transit("asked_learn", "done_learn", action="done_learn_gen")
ag.add_transit("done_learn", "asked_gen", action="ask_gen", args={"du_hash": "<playlist>", "ask_steps": 2000})
ag.add_transit("asked_gen", "done_gen", action="done_gen")
ag.add_state_action("done_gen", action="next_pref_stream")
ag.add_transit("done_gen", "stream_shared", action="check_pref_stream", args={"what": "not_first"})
ag.add_transit("done_gen", "finished", action="check_pref_stream", args={"what": "first"})

# adding agent to environment
env.add_agent(ag)

# creating student agent
ag = BasicAgent("Student", model=DemoModel(attributes=env.shared_attributes, lr=0.001, device=device), authority=0.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
ag.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
ag.add_transit("got_teacher_streams", "learning", action="do_learn_gen")
ag.add_transit("got_teacher_streams", "generated", action="do_gen")
ag.add_transit("learning", "got_teacher_streams", action="nop")
ag.add_transit("generated", "got_teacher_streams", action="nop")

# adding agent to environment
env.add_agent(ag)

# printing
print(env)
for ag in env.agents.values():
    print(ag)

# creating server
Server(env=env)

# running
env.run()

[Environment] env
	- agents:
		Teacher (authority: 1.0)
		Student (authority: 0.0)
	- streams:
		env:sin
	- behaviour:
		{
		    "initial_state": "init",
		    "state": "init",
		    "prev_state": null,
		    "limbo_state": null,
		    "state_actions": {
		        "init": [
		            null,
		            null,
		            0
		        ],
		        "streams_enabled": [
		            null,
		            null,
		            1
		        ],
		        "streams_sent": [
		            null,
		            null,
		            2
		        ],
		        "ready": [
		            null,
		            null,
		            3
		        ]
		    },
		    "transitions": {
		        "init": {
		            "streams_enabled": [
		                [
		                    "enable_all_streams",
		                    {},
		                    false,
		                    0
		                ]
		            ]
		        },
		        "streams_enabled": {
		            "streams_sent": [
		                [
		      

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.101.11.147:5001
Press CTRL+C to quit
10.101.11.147 - - [20/Mar/2025 09:41:26] "GET / HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:26] "GET /index_dist.css HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:26] "GET /static/css/main.aea32471.css HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:26] "GET /static/js/main.f41edc3f.js HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:28] "GET /get_env_name HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:28] "GET /get_list_of_agents?agent_name=env HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:28] "GET /get_play_pause_status HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:28] "GET /favicon.ico HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:29] "GET /get_list_of_streams?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:29] "GET /get_list_of_streams?agent_name=env HTTP/1.1" 200 -
10.101.1

[envir: env] >>> Clock step 0 <<<
[envir: env, state: init, act: enable_all_streams()] Enabling all streams


10.101.11.147 - - [20/Mar/2025 09:41:54] "GET /get_behav_status?agent_name=env HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:54] "GET /get_console?agent_name=env HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:54] "GET /get_console?agent_name=Teacher&last_only=true HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:54] "GET /get_behav_status?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:54] "GET /get_console?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:54] "GET /get_console?agent_name=Student&last_only=true HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:54] "GET /get_behav_status?agent_name=Student HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:54] "GET /get_console?agent_name=Student HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:55] "GET /get_list_of_streams?agent_name=env HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:41:55] "GET /get_list_of_streams?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.1

[envir: env] >>> Clock step 1 <<<
[envir: env, state: streams_enabled, act: send_streams_to_all()] Providing 1 streams to all agents
[agent: Teacher, state: init, act: get_streams(agent=None, streams={...})] Getting 1 streams from the environment
[agent: Student, state: init, act: get_streams(agent=None, streams={...})] Getting 1 streams from the environment


...

10.101.11.147 - - [20/Mar/2025 09:45:50] "GET /get_play_pause_status HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:45:51] "GET /get_console?agent_name=Teacher&last_only=true HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:45:51] "GET /get_behav_status?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:45:51] "GET /get_console?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:45:51] "GET /get_console?agent_name=Student&last_only=true HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:45:51] "GET /get_behav_status?agent_name=Student HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:45:51] "GET /get_console?agent_name=Student HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:45:51] "GET /get_list_of_agents?agent_name=env HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:45:51] "GET /get_list_of_streams?agent_name=env HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:45:51] "GET /get_list_of_streams?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 

[envir: env] >>> Clock step 4007 <<<
[agent: Teacher, state: asked_learn, act: done_learn_gen(agent=Student, streams={...})] Agent Student finished learning to generate


10.101.11.147 - - [20/Mar/2025 09:48:59] "GET /get_stream?agent_name=Student&stream_name=Student:generated1+[y]&since_step=4007 HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:48:59] "GET /get_stream?agent_name=Student&stream_name=Student:target1+[y]&since_step=4007 HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:48:59] "GET /get_console?agent_name=Teacher&last_only=true HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:48:59] "GET /get_behav_status?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:48:59] "GET /get_console?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:48:59] "GET /get_console?agent_name=Student&last_only=true HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:48:59] "GET /get_behav_status?agent_name=Student HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:48:59] "GET /get_console?agent_name=Student HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:48:59] "GET /get_list_of_streams?agent_name=env HTTP/1.1" 200 -
10.101.11.147

[envir: env] >>> Clock step 4008 <<<


10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_play_pause_status HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_list_of_agents?agent_name=env HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_console?agent_name=Teacher&last_only=true HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_behav_status?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_console?agent_name=Teacher HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_console?agent_name=Student&last_only=true HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_behav_status?agent_name=Student HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_console?agent_name=Student HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_stream?agent_name=Student&stream_name=Student:generated1+[y]&since_step=4007 HTTP/1.1" 200 -
10.101.11.147 - - [20/Mar/2025 09:49:01] "GET /get_stream?agent_name=S

## learn and generate a playlist
In this case, we are creating a Sandbox in which the Teacher has 2 different stream samples available for its Students.\
The lesson is structured as a playlist in which the Teacher provides the supervision, alternating the two signals.\
After 5 repetitions of each signal, the Student is aked to reproduce both of them.

In [ ]:
# creating environment
env = BasicEnvironment("env", title="Demo Sandbox Playlist")
device = torch.device("cpu")

# adding streams to the environment
env.add_stream(Stream.create(name="sin", creator=env.name, stream=Sin(freq=0.06, phase=0.5, delta=0.1)))
env.add_stream(Stream.create(name="square", creator=env.name, stream=Square(freq=0.06, ampl=0.5, phase=0.5, delta=0.1)))

# modeling behaviour of the environment
env.add_transit("init", "streams_enabled", action="enable_all_streams")
env.add_transit("streams_enabled", "streams_sent", action="send_streams_to_all")
env.add_transit("streams_sent", "ready", action="send_agents_to_all")

# creating teacher agent
ag = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "recording1", action="record", args={"stream_hash": env.name + ":sin", "steps": 500})
ag.add_transit("recording1", "recording2", action="record", args={"stream_hash": env.name + ":square", "steps": 500})
ag.add_transit("recording2", "playlist_ready", action="set_pref_streams",
               args={"stream_hashes": [ag.name + ":recorded1", ag.name + ":recorded2"], "repeat": 6})
ag.add_state_action("playlist_ready", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})
ag.add_transit("playlist_ready", "student_found", action="send_engagement")
ag.add_transit("student_found", "playlist_ready", action="nop")
ag.add_transit("student_found", "student_engaged", action="got_engagement")
ag.add_transit("student_engaged", "stream_shared", action="share_streams")
ag.add_transit("stream_shared", "asked_learn", action="ask_learn_gen",
               args={"du_hash": "<playlist>", "yhat_hash": "<playlist>", "dhat_hash": "<playlist>",
                     "ask_steps": 500})
ag.add_transit("asked_learn", "done_learn", action="done_learn_gen")
ag.behav.add_state_action("done_learn", action="next_pref_stream")
ag.behav.add_transit("done_learn", "stream_shared", action="check_pref_stream", args={"what": "not_last_round"})
ag.behav.add_transit("done_learn", "ready_to_ask", action="check_pref_stream", args={"what": "last_round"})
# add a final unsupervised generation for each signal
ag.behav.add_transit("ready_to_ask", "asked_gen", action="ask_gen",
                     args={"du_hash": "<playlist>",  "dhat_hash": "<playlist>", "ask_steps": 500})
ag.behav.add_transit("asked_gen", "done_gen", action="done_gen")
ag.behav.add_state_action("done_gen", action="next_pref_stream")
ag.behav.add_transit("done_gen", "ready_to_ask", action="check_pref_stream", args={"what": "not_first"})
ag.behav.add_transit("done_gen", "finished", action="check_pref_stream", args={"what": "first"})

# adding agent to environment
env.add_agent(ag)

# creating student agent
ag = BasicAgent("Student", model=BasicModel(attributes=env.shared_attributes, lr=0.001, device=device),
                authority=0.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
ag.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
ag.add_transit("got_teacher_streams", "learning", action="do_learn_gen_and_pred")
ag.add_transit("got_teacher_streams", "generated", action="do_gen_and_pred")
ag.add_transit("learning", "got_teacher_streams", action="nop")
ag.add_transit("generated", "got_teacher_streams", action="nop")

# adding agent to environment
env.add_agent(ag)

# printing
print(env)
for ag in env.agents.values():
    print(ag)

# creating server
Server(env=env)

# running
env.run()


## learn, generate playlist and descriptor

## old python code

#### basic_model_gen4all_pre4all_gd

In [ ]:
class DemoModel(Model):

    def __init__(self, attributes: list[Attributes], lr: float = 0.0001, device: torch.device = torch.device("cpu")):
        """Creates a model composed of a generator and a predictor."""

        # getting shape info from attributes (it is needed to build the generator/predictor)
        assert len(attributes) == 2, "Only two attributes are supported/expected (about y and d)"
        u_shape = attributes[0].shape
        d_dim = attributes[1].shape.numel()
        y_dim = attributes[0].shape.numel()

        generator = GenCTBEInitStateBZeroInput(u_shape=u_shape, d_dim=d_dim, y_dim=y_dim, h_dim=500, delta=0.1,
                                               local=False, cnu_memories=0)
        predictor = PredRNN(y_dim=y_dim, d_dim=d_dim, h_dim=10)

        # creating the model (superclass)
        super(BasicModel, self).__init__(generator, predictor, attributes, device=device)

        # extra stuff
        self.optim = torch.optim.SGD(list(self.generator.parameters()) + list(self.predictor.parameters()), lr=lr)
        self.loss_gen = torch.nn.functional.mse_loss
        self.loss_pred = torch.nn.functional.mse_loss

    def learn(self,
              y: torch.Tensor | None, yhat: torch.Tensor | None,
              d: torch.Tensor | None, dhat: torch.Tensor | None) \
            -> tuple[float, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None]:
        """Learn from different types of data (some of them could be None)."""

        # clean arguments, ensuring that what should be forced to None is actually forced to None
        _, y, yhat, d, dhat = super().learn(y, yhat, d, dhat)  # it seems unuseful, but IT MUST be called!

        # evaluating loss function
        loss = ((self.loss_gen(y, yhat) if yhat is not None else 0.) +
                (self.loss_pred(d, dhat) if dhat is not None else 0.))

        # learning
        self.optim.zero_grad()
        loss_as_float = loss.item()
        loss.backward()
        self.optim.step()

        return loss_as_float, y, yhat, d, dhat


#### sandbox_example

In [ ]:
import torch
from narnian.server import Server
from narnian.streams import Stream
from narnian.model import EmptyModel
from basic.basic_agent import BasicAgent
from basic.basic_model_gen4all_pred4all_gd import BasicModel
from basic.basic_streams import Sin, Square
from basic.basic_environment import BasicEnvironment

# creating environment
env = BasicEnvironment("env", title="Sample Sandbox")
device = torch.device("cpu")

# adding streams to the environment
env.add_stream(Stream.create(name="sin", creator=env.name, stream=Sin(freq=0.06, phase=0.5, delta=0.1)))
env.add_stream(Stream.create(name="square", creator=env.name, stream=Square(freq=0.06, ampl=0.5, phase=0.5, delta=0.1)))

# modeling behaviour of the environment
env.add_transit("init", "streams_enabled", action="enable_all_streams")
env.add_transit("streams_enabled", "streams_sent", action="send_streams_to_all")
env.add_transit("streams_sent", "ready", action="send_agents_to_all")

# creating teacher agent
ag = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "recording1", action="record",
               args={"stream_hash": env.name + ":sin", "steps": 500})
ag.add_transit("recording1", "recording2", action="record",
               args={"stream_hash": env.name + ":square", "steps": 500})
ag.add_transit("recording2", "playlist_ready", action="set_pref_streams",
               args={"stream_hashes": [ag.name + ":recorded1", ag.name + ":recorded2"], "repeat": 1})
ag.add_state_action("playlist_ready", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})
ag.add_transit("playlist_ready", "student_found", action="send_engagement")
ag.add_transit("student_found", "playlist_ready", action="nop")
ag.add_transit("student_found", "student_engaged", action="got_engagement")
ag.add_transit("student_engaged", "stream_shared", action="share_streams")
ag.add_transit("stream_shared", "asked_learn", action="ask_learn_gen_and_pred",
               args={"du_hash": "<playlist>", "yhat_hash": "<playlist>", "dhat_hash": "<playlist>",
                     "ask_steps": 500})
ag.add_transit("asked_learn", "done_learn", action="done_learn_gen_and_pred")
ag.add_transit("done_learn", "asked_gen", action="ask_gen_and_pred",
               args={"du_hash": "<playlist>", "ask_steps": 500})
ag.add_transit("asked_gen", "done_gen", action="done_gen_and_pred")
ag.add_state_action("done_gen", action="eval",
                    args={"stream_hash": "<playlist>", "what": "y", "how": "mse", "steps": 500})
ag.add_transit("done_gen", "stream_shared", action="compare_eval", args={"cmp": ">", "thres": 0.05})
ag.add_transit("done_gen", "good", action="compare_eval", args={"cmp": "<=", "thres": 0.05})
ag.add_state_action("good", action="next_pref_stream")
ag.add_transit("good", "stream_shared", action="check_pref_stream", args={"what": "not_first"})
ag.add_transit("good", "finished", action="check_pref_stream", args={"what": "first"})

# adding agent to environment
env.add_agent(ag)

# creating student agent
ag = BasicAgent("Student", model=BasicModel(attributes=env.shared_attributes, lr=0.001, device=device),
                authority=0.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
ag.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
ag.add_transit("got_teacher_streams", "learning", action="do_learn_gen_and_pred")
ag.add_transit("got_teacher_streams", "generated", action="do_gen_and_pred")
ag.add_transit("learning", "got_teacher_streams", action="nop")
ag.add_transit("generated", "got_teacher_streams", action="nop")

# adding agent to environment
env.add_agent(ag)

# printing
print(env)
for ag in env.agents.values():
    print(ag)

# creating server
Server(env=env)

# running
env.run()
